In [26]:
from fighters import Missile, Target, LineOfSight, simulation
from guidance import pn
import numpy as np
from math import *

In [27]:
los_opts = np.array([4e4, np.radians(20), np.radians(45)])
los_opts;

In [28]:
target_opts = {
        'x0': 0,
        'y0': 15e3,
        'z0': 0,
        'vel0': 400,
        'theta0': np.radians(20),
        'psi0': np.radians(-70) }
target_opts;

In [29]:
missile_opts = {
    'initial_state': {
        'x0': target_opts['x0'] + los_opts[0]*cos(los_opts[1])*cos(los_opts[2]),
        'y0': target_opts['y0'] + los_opts[0]*sin(los_opts[1]),
        'z0': target_opts['z0'] + los_opts[0]*cos(los_opts[1])*sin(los_opts[2]),
        'vel0': 400,
        'theta0': pi + los_opts[1] + np.radians(10),
        'psi0': los_opts[2] },
    'energetics': {
        'mass0': 110,
        'omega_act': 45,
        'omega_march': 0,
        't_act': 5,
        't_march': 0,
        'I_1': 1800},
    'aerodynamics': {
        'D': 160e-3,
        'spatial_angle_max': np.radians(40),
        'Ms': np.array([0.1, 0.3, 0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1., 1.05, 
                        1.1, 1.15, 1.5, 1.9, 2.3, 2.7, 3.1, 3.5, 3.9, 4.3, 4.7]),
        'Cx0_arr': np.array([0.2958, 0.2998, 0.3078, 0.3198, 0.3234, 0.3273, 0.3314, 0.3358, 0.3404, 0.3453, 0.3504,
                             0.3558, 0.3614, 0.4426, 0.3598, 0.2885, 0.2376, 0.2009, 0.1741, 0.1533, 0.1377, 0.1255]),
        'Cya_arr': np.array([0.0571, 0.0577, 0.0591, 0.0615, 0.0623, 0.0633, 0.0643, 0.0655, 0.0671, 0.0708, 0.0794,
                             0.0814, 0.0802, 0.0664, 0.0543, 0.0465, 0.042, 0.0387, 0.0358, 0.0331, 0.0312, 0.0291]),
        'Cyb_arr': np.array([0.0571, 0.0577, 0.0591, 0.0615, 0.0623, 0.0633, 0.0643, 0.0655, 0.0671, 0.0708, 0.0794,
                             0.0814, 0.0802, 0.0664, 0.0543, 0.0465, 0.042, 0.0387, 0.0358, 0.0331, 0.0312, 0.0291]) }
}
missile_opts;

In [30]:
def target_autopilot(*args):
    return np.radians(4), np.radians(0)

In [31]:
%%time
res = simulation(missile_opts, target_opts, target_autopilot, pn, 10, t_integration=100)
print(f"{len(res['t'])} iterations;\nr = {round(res['r'][-1])} м")

Мы попали
4221 iterations;
r = 27.0 м
Wall time: 5.39 s


In [32]:
%matplotlib
import matplotlib.pyplot as plt
plt.plot(res['t'], res['r'])

Using matplotlib backend: Qt5Agg


In [33]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "browser"

In [34]:
h = 20

# data
data = go.Scatter3d(x=[res['missile_x'][0]], 
                    y=[res['missile_z'][0]], 
                    z=[res['missile_y'][0]],
                    marker=dict(size=1), 
                    name="rocket trajectory")
datac = go.Scatter3d(x=[res['target_x'][0]], 
                     y=[res['target_z'][0]], 
                     z=[res['target_y'][0]],
                     marker=dict(size=1), 
                     name="target trajectory")
connect_line = go.Scatter3d(x=[data['x'][0],datac['x'][0]], 
                            y=[data['y'][0],datac['y'][0]], 
                            z=[data['z'][0],datac['z'][0]],
                            marker=dict(size=1), 
                            name="connect line")
v_cone = go.Cone(x=[res['missile_x'][0]], 
            y=[res['missile_z'][0]], 
            z=[res['missile_y'][0]],
            u=[res['missile_vel'][0]*cos(res['missile_theta'][0])*cos(res['missile_psi'][0])], 
            v=[res['missile_vel'][0]*cos(res['missile_theta'][0])*sin(res['missile_psi'][0])], 
            w=[res['missile_vel'][0]*sin(res['missile_theta'][0])],
            sizemode='absolute',
            sizeref=2e3,
            showscale=False, 
            colorscale=[[0, 'blue'], [1., 'blue']])
vc_cone = go.Cone(x=[res['target_x'][0]], 
             y=[res['target_z'][0]], 
             z=[res['target_y'][0]],
             u=[res['target_vel'][0]*cos(res['target_theta'][0])*cos(res['target_psi'][0])], 
             v=[res['target_vel'][0]*cos(res['target_theta'][0])*sin(res['target_psi'][0])], 
             w=[res['target_vel'][0]*sin(res['target_theta'][0])],
             sizemode='absolute',
             sizeref=2e3,
             showscale=False, 
             colorscale=[[0, 'red'], [1., 'red']])
v = go.Scatter(x=[res['t'][0]], 
               y=[res['missile_vel'][0]],
               marker=dict(size=1), 
               name="rocket velocity", 
               xaxis="x2", 
               yaxis="y2")
vc = go.Scatter(x=[res['t'][0]], 
                y=[res['target_vel'][0]],
                marker=dict(size=1), 
                name="target velocity", 
                xaxis="x2", 
                yaxis="y2")
alpha = go.Scatter(x=[res['t'][0]], 
                   y=[57.3*res['missile_alpha'][0]],
                   marker=dict(size=1), 
                   name="rocket alpha", 
                   xaxis="x3", 
                   yaxis="y3")
beta = go.Scatter(x=[res['t'][0]], 
                  y=[57.3*res['missile_beta'][0]],
                  marker=dict(size=1), 
                  name="rocket beta", 
                  xaxis="x3", 
                  yaxis="y3")
missile_spatial_angle = go.Scatter(x=[res['t'][0]], 
                                   y=[57.3*res['missile_spatial_angle'][0]],
                                   marker=dict(size=1), 
                                   name="rocket spatial angle", 
                                   xaxis="x3", 
                                   yaxis="y3")

data = [data, datac, connect_line, v_cone, vc_cone, v, vc, alpha, beta, missile_spatial_angle]

# layout
layout = go.Layout(scene=dict(xaxis=dict(nticks=6, 
                                         range=[-50e3,50e3],
                                         zeroline=True,
                                         zerolinecolor="black",
                                         zerolinewidth=3,
                                         backgroundcolor="rgb(250, 250, 250)",
                                         gridcolor="lightgrey",
                                         showbackground=True, 
                                         title="x, м"),
                              yaxis=dict(nticks=6, 
                                         range=[-50e3,50e3],
                                         zeroline=True,
                                         zerolinecolor="black",
                                         zerolinewidth=3,
                                         backgroundcolor="rgb(250, 250, 250)",
                                         gridcolor="lightgrey",
                                         showbackground=True, 
                                         title="z, м"),
                              zaxis=dict(nticks=6, 
                                         range=[0,100e3],
                                         zeroline=False,
                                         zerolinecolor="black",
                                         zerolinewidth=3,
                                         backgroundcolor="rgb(250, 250, 250)",
                                         gridcolor="lightgrey",
                                         showbackground=True, 
                                         title="h, м"),
                              domain=dict(column=0,
                                          row=0,
                                          x=[0, 1],
                                          y=[0, 0.75]),
                              aspectmode='cube'
                             ),
                   xaxis2=dict(domain=[0.6, 1.], 
                               range=[0,res['t'][-1]],
                               nticks=6,
                               zeroline=False,
                               zerolinecolor="black",
                               zerolinewidth=3, 
                               gridcolor="lightgrey", 
                               linecolor="black", 
                               title="t, с", 
                               anchor="y2"),
                   yaxis2=dict(domain=[0.8, 1.], 
                               range=[0,max(res['missile_vel'])+100],
                               nticks=6,
                               zeroline=False,
                               zerolinecolor="black",
                               zerolinewidth=3, 
                               gridcolor="lightgrey", 
                               linecolor="black", 
                               title="v, м/с", 
                               anchor="x2"),
                   xaxis3=dict(domain=[0.07, 0.47], 
                               range=[0,res['t'][-1]],
                               nticks=6,
                               zeroline=False,
                               zerolinecolor="black",
                               zerolinewidth=3, 
                               gridcolor="lightgrey", 
                               linecolor="black", 
                               title="t, с", 
                               anchor="y3"),
                   yaxis3=dict(domain=[0.8, 1.], 
                               range=[57.3*min(min(res['missile_alpha']), 
                                               min(res['missile_beta']),
                                               min(res['missile_spatial_angle'])), 
                                      57.3*max(max(res['missile_alpha']), 
                                               max(res['missile_beta']),
                                               max(res['missile_spatial_angle']))],
                               nticks=6,
                               zeroline=False,
                               zerolinecolor="black",
                               zerolinewidth=0.1, 
                               gridcolor="lightgrey", 
                               linecolor="black", 
                               title="alpha, beta, \nsp_angle °", 
                               anchor="x3"),
                   plot_bgcolor="rgb(250, 250, 250)",
                   width=1000,
                   height=1000,
                   margin=dict(r=10, l=10, b=10, t=10),
                   updatemenus=[dict(type="buttons",
                                     buttons=[dict(label="Play",
                                                   method="animate",
                                                   args=[None, {"frame": {"duration": 0, "redraw": True}, 
                                                                "fromcurrent": True}]), 
                                              dict(label="Pause",
                                                   method="animate",
                                                   args=[[None], {"frame": {"duration": 0, "redraw": True},
                                                                  "mode": "immediate",
                                                                  "transition": {"duration": 0}}])])]
                  )

# frames
frames=[go.Frame(
    data=[go.Scatter3d(
        x=res['missile_x'][::h][:i], 
        y=res['missile_z'][::h][:i], 
        z=res['missile_y'][::h][:i], 
        mode='lines', 
        marker=dict(size=1)),
          go.Scatter3d(
        x=res['target_x'][::h][:i], 
        y=res['target_z'][::h][:i], 
        z=res['target_y'][::h][:i], 
        mode='lines', 
        marker=dict(size=1)),
          go.Scatter3d(
        x=[res['missile_x'][::h][i], res['target_x'][::h][i]], 
        y=[res['missile_z'][::h][i], res['target_z'][::h][i]], 
        z=[res['missile_y'][::h][i], res['target_y'][::h][i]], 
        mode='lines', 
        marker=dict(size=1)),
          go.Cone(
        x=[res['missile_x'][::h][i]], 
        y=[res['missile_z'][::h][i]], 
        z=[res['missile_y'][::h][i]],
        u=[res['missile_vel'][::h][i]*cos(res['missile_theta'][::h][i])*cos(res['missile_psi'][::h][i])], 
        v=[res['missile_vel'][::h][i]*cos(res['missile_theta'][::h][i])*sin(res['missile_psi'][::h][i])], 
        w=[res['missile_vel'][::h][i]*sin(res['missile_theta'][::h][i])],
        sizemode='absolute',
        sizeref=2e3),
          go.Cone(
        x=[res['target_x'][::h][i]], 
        y=[res['target_z'][::h][i]], 
        z=[res['target_y'][::h][i]],
        u=[res['target_vel'][::h][i]*cos(res['target_theta'][::h][i])*cos(res['target_psi'][::h][i])], 
        v=[res['target_vel'][::h][i]*cos(res['target_theta'][::h][i])*sin(res['target_psi'][::h][i])], 
        w=[res['target_vel'][::h][i]*sin(res['target_theta'][::h][i])],
        sizemode='absolute',
        sizeref=2e3),
          go.Scatter(
        x=res['t'][::h][:i], 
        y=res['missile_vel'][::h][:i],
        mode='lines',
        marker=dict(size=1), 
        xaxis="x2", 
        yaxis="y2"),
          go.Scatter(
        x=res['t'][::h][:i], 
        y=res['target_vel'][::h][:i],
        mode='lines',
        marker=dict(size=1), 
        xaxis="x2", 
        yaxis="y2"),
          go.Scatter(
        x=res['t'][::h][:i], 
        y=57.3*res['missile_alpha'][::h][:i],
        mode='lines',
        marker=dict(size=1), 
        xaxis="x3", 
        yaxis="y3"),
          go.Scatter(
        x=res['t'][::h][:i], 
        y=57.3*res['missile_beta'][::h][:i],
        mode='lines',
        marker=dict(size=1), 
        xaxis="x3", 
        yaxis="y3"),
          go.Scatter(
        x=res['t'][::h][:i], 
        y=57.3*res['missile_spatial_angle'][::h][:i],
        mode='lines',
        marker=dict(size=1), 
        xaxis="x3", 
        yaxis="y3"),
         ]) 
    for i in range(len((res['missile_x'][::h])))]

fig = go.Figure(data=data, layout=layout, frames=frames)

fig.show()